In [46]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver #not used

import time #not used
from selenium.webdriver.common.keys import Keys #not used

###### FOR HEROKU
#import os
#op = webdriver.ChromeOptions()
#op.binary_location = os.environ.get("GOOGLE_CHROME_BIN")
#op.add_argument("--headless")
#op.add_argument("--no-sandbox")
#op.add_argument("--disable-dev-sh-usage")
#HEADERS ={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

In [11]:
def get_test(url, job_title, location):
    driver = webdriver.Chrome("/Users/dswiss/Desktop/chromedriver")
    ### heroku webdriver
    #driver = webdriver.Chrome(executable_path=os.environ.get("CHROMEDRIVER_PATH"), options=op)
    driver.get(url)
    time.sleep(3)
    driver.find_element_by_xpath('//*[@id="text-input-what"]').send_keys(job_title)
    time.sleep(3)
    locations = driver.find_element_by_xpath('//*[@id="text-input-where"]')
    if locations.get_attribute("value") == "":
        driver.find_element_by_xpath('//*[@id="text-input-where"]').send_keys(location)
    time.sleep(3)
    driver.find_element_by_xpath('/html/body/div').click()
    time.sleep(3)
    try:
        driver.find_element_by_xpath('//*[@id="jobsearch"]/button').click()
    except:
        driver.find_element_by_xpath('//*[@id="whatWhereFormId"]/div[3]/button').click()
    current_url = driver.current_url
    return current_url
    
current_url = get_test('https://indeed.com/','Data Scientist',"Los Angeles")

In [65]:
def scrape_job_details(url):
    
    resp = requests.get(url)
    content = BeautifulSoup(resp.content, 'lxml')
   
    jobs_list = []    
    for post in content.select('.job_seen_beacon'):
        try:
            data = {
                "job_title":post.select('.jobTitle')[0].get_text().strip(),
                "company":post.select('.companyName')[0].get_text().strip(),
                "rating":post.select('.ratingNumber')[0].get_text().strip(),
                "location":post.select('.companyLocation')[0].get_text().strip(),
                "date":post.select('.date')[0].get_text().strip(),
                "job_desc":post.select('.job-snippet')[0].get_text().strip(),
                "job_link":post.find(class_='jcs-JobTitle', href=True)['href']
            }
        except IndexError:
            continue          
        jobs_list.append(data)
    dataframe = pd.DataFrame(jobs_list)
    dataframe["job_link"] = 'https://www.indeed.com' + dataframe['job_link'].astype(str)
     
    return dataframe

df = scrape_job_details(current_url)
df["date"] = [x.replace('Posted','') for x in df["date"]]
df["job_title"] = [x.replace('new', '') for x in df["job_title"]]

In [67]:
lol

[<div class="job_seen_beacon"><table cellpadding="0" cellspacing="0" class="jobCard_mainContent big6_visualChanges" role="presentation"><tbody><tr><td class="resultContent"><div class="heading4 color-text-primary singleLineTitle tapItem-gutter"><h2 class="jobTitle jobTitle-color-purple jobTitle-newJob" tabindex="-1"><div class="new topLeft holisticNewBlue desktop"><span class="label">new</span></div><a aria-label="full details of Senior Data Science Manager - Jobseeker Discovery &amp; Communic..." class="jcs-JobTitle" data-ci="383080872" data-empn="9767451333976163" data-hide-spinner="true" data-hiring-event="false" data-jk="5aa51ec592b942da" data-mobtk="1g50td1ggj5le800" href="/pagead/clk?mo=r&amp;ad=-6NYlbfkN0CiRNM7CVr8YueLFKlzwbFWI0o7IjV438l4sVrvKZ0flpURU_mqoI8E-VxPfg2eTCH8IGCEHSQlfNSAvIB_2NPat3Ixrj0z_JOqgSnnsyWM1iW0IslGNMGFhhySqQ9j86Z2BPOEg7QcrzFIyduuGKZ94hdLLX-SJ1Vbzl7Z7NBAowwOI5YWEXIQ9xx3NP6w0eB0bXSF6P_CtypLLr0hYTzoWEB6yN_6RszYXhEifuS_5T_ngQVDXD73Sc8uvjYbaa2p-ftj8gsJUpOstjieeEJEe

In [ ]:
### FOR OUR ACTUAL HEROKUAPP

In [60]:
user_job = "Data Science"
user_location = "New York"

user_job = user_job.replace(" ", "%20")
user_location = user_location.replace(" ", "%20")

current_url = f"https://www.indeed.com/jobs?q={user_job}&l={user_location}"

In [61]:
current_url

'https://www.indeed.com/jobs?q=Data%20Science&l=New%20York'

In [55]:
lol

,job_title,company,rating,location,date,job_desc,job_link
0,"Data Scientist, Spotify Advertising",Spotify,4.3,"Remote in New York, NY+7 locations",30+ days ago,You have experience in a similar data science ...,https://www.indeed.com/rc/clk?jk=cf1dfad666491...
1,Data Scientist - Data Science (All Levels),Datadog,4.0,"Hybrid remote in New York, NY+3 locations",26 days ago,You have experience working with data and buil...,https://www.indeed.com/rc/clk?jk=3f72ccd78579c...
2,Data Scientist,Nike,4.1,"New York, NY",6 days ago,Identify meaningful insights from large data a...,https://www.indeed.com/rc/clk?jk=f4f924ccdcaf1...
3,Data Science Rotation Program Associate,New York Life Insurance Co,3.7,"New York, NY+1 location",30+ days ago,You will be inspired to strengthen other criti...,https://www.indeed.com/rc/clk?jk=61c19d3402d03...
4,Data Scientist,Source Enterprises,4.3,"New York, NY 10036 (Midtown area)",30+ days ago,Some knowledge of common data science toolkits...,https://www.indeed.com/rc/clk?jk=1636faa0308fd...
5,Data Scientist,Signifyd,4.1,"Remote in New York, NY",5 days ago,Work with distributed data pipelines.\nDesign ...,https://www.indeed.com/company/Signifyd/jobs/D...
6,Data Analyst,DataDog,4.0,"New York, NY",28 days ago,Distill that data to concise insights and comm...,https://www.indeed.com/rc/clk?jk=51494a0a3e768...
7,Data Scientist - Analytics,Twitch,4.4,"Remote in New York, NY 10011",30+ days ago,5+ years of experience as a data scientist or ...,https://www.indeed.com/rc/clk?jk=437bdb20ea775...
8,Head of Advertising & Marketing - Data Science,"Amazon Web Services, Inc.",3.5,"New York, NY+1 location",30+ days ago,Experience with applying CI/CD principles to d...,https://www.indeed.com/rc/clk?jk=6d1ef76bf3b23...


In [14]:
lol.iloc[6]["job_link"]

'https://www.indeed.com/rc/clk?jk=ba6f99e6f2792075&fccid=f2d0ffe36bb2f0f4&vjs=3'

In [57]:
table = lol.to_html()

In [ ]:
### forces download
# text_file = open("index.html", "w")
# text_file.write(html)
# text_file.close()

In [58]:
table = table.replace("\n", "")

In [59]:
table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>job_title</th>      <th>company</th>      <th>rating</th>      <th>location</th>      <th>date</th>      <th>job_desc</th>      <th>job_link</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Data Scientist, Spotify Advertising</td>      <td>Spotify</td>      <td>4.3</td>      <td>Remote in New York, NY+7 locations</td>      <td>30+ days ago</td>      <td>You have experience in a similar data science role and a degree in science, computer science, statistics, economics, mathematics, or similar quantitative…</td>      <td>https://www.indeed.com/rc/clk?jk=cf1dfad66649138a&amp;fccid=fe404d18bb9eef1e&amp;vjs=3</td>    </tr>    <tr>      <th>1</th>      <td>Data Scientist - Data Science (All Levels)</td>      <td>Datadog</td>      <td>4.0</td>      <td>Hybrid remote in New York, NY+3 locations</td>      <td>26 days ago</td>      <td>You have experience working with data a